In [1]:
import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os
import pandas as pd

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

from copy import deepcopy
import boto3
from botocore.handlers import disable_signing

import datetime as dt
import pytz

Populating the interactive namespace from numpy and matplotlib

## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/icrisologo/.conda/envs/pyart/lib/python3.7/site-packages/botocore/awsrequest.py:624: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class HeadersDict(collections.MutableMapping):


In [2]:
def get_radar_field(radar):

    gatefilter = pyart.filters.GateFilter(radar)
    gatefilter.exclude_transition()
    gatefilter.exclude_masked('reflectivity')
    # Mask reflectivity
    radar.fields["corrected_reflectivity"] = deepcopy(radar.fields["reflectivity"])
    radar.fields["corrected_reflectivity"]["data"] = np.ma.masked_where(
        gatefilter._gate_excluded, radar.fields["corrected_reflectivity"]["data"])
    rr = pyart.retrieve.est_rain_rate_z(radar, refl_field="corrected_reflectivity")
    radar.add_field('rainrate',rr,replace_existing=True)
    # Mask out last 10 gates of each ray, this removes the "ring" around the radar.
    radar.fields['rainrate']['data'][:, -10:] = np.ma.masked
    
    return radar,gatefilter

# Load the AR database to get the dates

In [6]:
ARdatafile = '/lcrc/group/earthscience/icrisologo/DATA/ARcatalog_NCEP_NEW_1948-2019.csv'

Filter the database based on year and location (altitude)

In [7]:
df = pd.read_csv(ARdatafile)

year = 1997
df_ca = df.loc[(df.Lat > 35) & (df.Lat < 40) & (df.Year == year)] # filtering location, selecting only those in central CA

Get the list of dates where AR is present.

In [8]:
ar_dates = []
for m in df_ca.Month.unique():
    df_ = df_ca.loc[(df_ca.Month == m)]
    df_start = df_.iloc[[0]]
    dstart = df_start.Day.get_values()[0]
    df_end = df_.iloc[[-1]]
    dend = df_end.Day.get_values()[0]
    start = dt.datetime(year,m,dstart)
    end = dt.datetime(year,m,dend)
    ar_dates.append([start,end])

/home/icrisologo/.conda/envs/pyart/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  """
/home/icrisologo/.conda/envs/pyart/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  import sys


In [ ]:
for j in range(len(ar_dates)):
    d0 = ar_dates[j][0]
    d1 = ar_dates[j][1]
    td = dt.timedelta(days = 1)
    dates = np.arange(d0, d1, td).astype(dt.datetime)
    for d in dates:
        print(d)
        filepath = '/lcrc/group/earthscience/icrisologo/DATA/nexrad/'+str(d.year)+'/'+str(d.month).zfill(2)+'/'+str(d.day).zfill(2)

        fnamelist = os.listdir(filepath)
        fnamelist.sort()
        try:
            fname = fnamelist[0]
        except IndexError:
            continue
        radar = pyart.io.read(os.path.join(filepath,fname))
        rad,gatefilter = get_radar_field(radar)

        # setup gatefilter for sector --------------
        ranges = deepcopy(radar.fields['reflectivity'])
        azs = deepcopy(radar.fields['reflectivity'])

        #use matrix multplication to make 2d azimuth array
        azplus1 = np.expand_dims(radar.azimuth['data'], axis=0).T
        rangeplus1 = np.expand_dims(radar.range['data']*0.0 + 1., axis=0)
        az_2d = np.matmul(azplus1, rangeplus1)

        #use matrix multplication to make 2d range array
        azplus1 = np.expand_dims(radar.azimuth['data']*0.0 + 1., axis=0).T
        rangeplus1 = np.expand_dims(radar.range['data'], axis=0)
        r_2d = np.matmul(azplus1, rangeplus1)

        ranges['data'] = r_2d
        azs['data'] = az_2d
        radar.add_field('range2d', ranges, replace_existing=True)
        radar.add_field('az2d', azs, replace_existing=True)

        rlat = radar.latitude['data'][0]
        rlon = radar.longitude['data'][0]
        sweep = 0

        # make the gatefilter for sea sector
        mygf_sea = pyart.filters.GateFilter(radar)
        mygf_sea.exclude_all()
        mygf_sea.include_inside('az2d', 200,290)

        # make the gatefilter for the land sector
        mygf_land = pyart.filters.GateFilter(radar)
        mygf_land.exclude_all()
        mygf_land.include_inside('az2d', 20,110)

        # set up QVP arrays
        qvp_sea_refl = []
        qvp_sea_height = []
        qvp_sea_time = []
        qvp_sea_range = []

        qvp_land_refl = []
        qvp_land_height = []
        qvp_land_time = []
        qvp_land_range = []

        for fname in fnamelist:
            try:
                radar = pyart.io.read(os.path.join(filepath,fname))
            except:
                continue
            radar,gatefilter = get_radar_field(radar)

            #qvp = pyart.retrieve.quasi_vertical_profile(rad, fields='reflectivity', desired_angle=10.0, gatefilter=gatefilter)
            # generate QVPs
            try:
                qvp_sea = pyart.retrieve.quasi_vertical_profile(radar, fields='reflectivity', desired_angle=10.0, gatefilter=mygf_sea)
                qvp_land = pyart.retrieve.quasi_vertical_profile(radar, fields='reflectivity', desired_angle=10.0, gatefilter=mygf_land)
            except IndexError:
                continue

            timeobj = dt.datetime.strptime(qvp_sea['time']['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')

            qvp_sea_refl.append(qvp_sea['reflectivity'].data)
            qvp_sea_height.append(qvp_sea['height'].data)
            qvp_sea_range.append(qvp_sea['range'].data)  
            qvp_sea_time.append(timeobj)

            qvp_land_refl.append(qvp_land['reflectivity'].data)
            qvp_land_height.append(qvp_land['height'].data)
            qvp_land_range.append(qvp_land['range'].data)  
            qvp_land_time.append(timeobj)

        qvp_sea_refl = np.array(qvp_sea_refl)
        qvp_sea_height = np.array(qvp_sea_height)
        qvp_sea_time = np.array(qvp_sea_time)
        qvp_sea_range = np.array(qvp_sea_range)

        qvp_land_refl = np.array(qvp_land_refl)
        qvp_land_height = np.array(qvp_land_height)
        qvp_land_time = np.array(qvp_land_time)
        qvp_land_range = np.array(qvp_land_range)

        # create dictionary
        data = {}
        #
        data['qvp_sea_reflectivity'] = qvp_sea_refl
        data['qvp_sea_time'] = qvp_sea_time
        data['qvp_sea_height'] = qvp_sea_height
        data['qvp_sea_range'] = qvp_sea_range
        #
        data['qvp_land_reflectivity'] = qvp_land_refl
        data['qvp_land_time'] = qvp_land_time
        data['qvp_land_height'] = qvp_land_height
        data['qvp_land_range'] = qvp_land_range

        savefname = 'qvp_sector_10deg_'+str(timeobj.year)+str(timeobj.month).zfill(2)+str(timeobj.day).zfill(2)+'.hdf5'
        np.save(savefname, data)    

3
[datetime.datetime(1996, 4, 1, 0, 0) datetime.datetime(1996, 4, 2, 0, 0)
 datetime.datetime(1996, 4, 3, 0, 0) datetime.datetime(1996, 4, 4, 0, 0)
 datetime.datetime(1996, 4, 5, 0, 0) datetime.datetime(1996, 4, 6, 0, 0)
 datetime.datetime(1996, 4, 7, 0, 0) datetime.datetime(1996, 4, 8, 0, 0)
 datetime.datetime(1996, 4, 9, 0, 0) datetime.datetime(1996, 4, 10, 0, 0)
 datetime.datetime(1996, 4, 11, 0, 0) datetime.datetime(1996, 4, 12, 0, 0)
 datetime.datetime(1996, 4, 13, 0, 0) datetime.datetime(1996, 4, 14, 0, 0)
 datetime.datetime(1996, 4, 15, 0, 0) datetime.datetime(1996, 4, 16, 0, 0)
 datetime.datetime(1996, 4, 17, 0, 0) datetime.datetime(1996, 4, 18, 0, 0)
 datetime.datetime(1996, 4, 19, 0, 0) datetime.datetime(1996, 4, 20, 0, 0)
 datetime.datetime(1996, 4, 21, 0, 0) datetime.datetime(1996, 4, 22, 0, 0)
 datetime.datetime(1996, 4, 23, 0, 0)]
1996-04-01 00:00:00
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.9  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.9  9.9 14.6 19.5]

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.5  1.5  2.4  3.3  4.3  6.   9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.5  1.5  2.4  3.3  4.3  6.   9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.8  9.9 14.6 19.5]
19.51

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.1  9.9 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.1  9.9 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.3  1.5  2.4  3.4  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.3  1.5  2.4  3.4  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.8 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.8 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.7 19.5]
19.46

[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  6.1  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.8 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.7 19.5]
19.46

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.9 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.9 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.2  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.2  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.3  3.3  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.3  3.3  4.3  5.8  9.9 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.7 19.5]
19.46

...

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.3  4.3  5.9 10.  14.6 19.5]
19.51

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.9  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.9  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.8  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.8  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.5  1.5  2.4  3.4  4.4  6.1  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.5  1.5  2.4  3.4  4.4  6.1  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.3  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.3  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.6 19.5]
19.51

[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8 10.  14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8 10.  14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.8 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.8 10.  14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.4  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.9 14.7 19.5]
19.51

[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.5 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.8  9.9 14.5 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  5.8  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.4  6.   9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.5  1.5  2.4  3.3  4.3  5.8  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.5  1.5  2.4  3.3  4.3  5.8  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.8 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.8 14.7 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.3  4.3  5.9  9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.3  5.9  9.8 14.6 19.5]
19.46

[ 0.5  0.5  1.4  1.5  2.3  3.3  4.3  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.3  3.3  4.3  5.8  9.8 14.6 19.5]
19.467773
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.3  5.9  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.   9.9 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.4  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.2  5.8  9.8 14.6 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.4  1.5  2.5  3.4  4.4  6.1  9.9 14.7 19.5]
19.511719
[ 0.5  0.5  1.5  1.5  2.4  3.4  4.4  6.   9.8 14.7 19.5]
19.51